In [2]:
import pandas as pd

# Load party database
party_df_74 = pd.read_csv("data/processed/pr_votes_long_2074.csv")
party_df_79 = pd.read_csv("data/processed/pr_votes_long_2079.csv")

In [3]:
dist_2074 = pd.read_csv("data/raw/district_wise_pr_result_statistics_2074.csv", encoding="utf-8-sig")

In [4]:
dist_2079 = pd.read_csv("data/raw/district_wise_result_statistics_2079.csv", encoding="utf-8-sig")

In [7]:
final_df = pd.concat([party_df_74[["district_id","party_id","election_year","Vote_Share"]], party_df_79[["district_id","party_id","election_year","Vote_Share"]]], ignore_index=True)

In [15]:
final_df = final_df.sort_values(
    by=["district_id", "party_id", "election_year"]
).reset_index(drop=True)

In [13]:
final_df["party_id"] = (
    "P" +
    final_df["party_id"]
      .astype(str)
      .str.extract(r"(\d+)", expand=False)
      .astype(int)
      .astype(str)
      .str.zfill(3)            # pad to 3 digits
)

In [19]:
final_df["vote_share_lag"] = (
    final_df
    .groupby(["district_id", "party_id"])["Vote_Share"]
    .shift(1)
)

In [21]:
final_df["vote_share_change"] = (
    final_df["Vote_Share"] - final_df["vote_share_lag"]
)


In [23]:
final_df["is_new_party"] = final_df["vote_share_lag"].isna().astype(int)


In [25]:
final_df["vote_share_lag"] = final_df["vote_share_lag"].fillna(0)
final_df["vote_share_change"] = final_df["vote_share_change"].fillna(0)

In [27]:
df_model_base = final_df[final_df["election_year"].isin([2079, 2082])].copy()


In [29]:
census = pd.read_csv("data/raw/district_wise_demographics_2078.csv")

In [30]:
dist_ref= pd.read_csv("data/reference/district.csv")

In [32]:
census = census.merge(
    dist_ref[["district_id", "DISTRICT"]],
    on="DISTRICT",
    how="left"
)

In [33]:
df_model_base = df_model_base.merge(
    census,
    left_on="district_id",
    right_on="district_id",
    how="left"
)

In [36]:
df_model_base["female_ratio"] = df_model_base["Female 2078"] / df_model_base["Population NPHC 2078"]

In [38]:
df_model_base.drop(columns=["DISTRICT","district","Population NPHC 2078", "Male 2078","Female 2078","Population aged 5 years & above"], inplace=True)

In [41]:
df_model_base.rename(columns={"Vote_Share": "vote_share"}, inplace=True)

In [48]:
party = pd.read_csv("data/raw/Party_Database.csv", encoding="utf-8-sig")

In [57]:
party_lin = pd.read_csv("data/reference/party_lineage.csv", encoding="utf-8-sig")

In [58]:
party_lin["party_id"] = (
    "P"
    + party_lin["party_id"]
        .astype(str)
        .str.extract(r"(\d+)", expand=False)
        .astype(int)
        .astype(str)
        .str.zfill(3)
)

In [174]:
party.to_csv("data/raw/Party_Database_79.csv", encoding="utf-8-sig",index=False)

In [61]:
df_model_base = df_model_base.merge(
    party_lin[[
        "party_id",
        "IS_Major?",
        "IS_ALTERNATIVE_FORCE?",
        "GenZ_and_Youth_Favored?",
        "Was_Part_Of_Ousted_Government?"
    ]],
    on="party_id",
    how="left"
)

In [73]:
df_model_base = df_model_base.merge(
    party[[
        "party_id",
        "Facebook_Presence (In Thousands)", 
        "Top_leader_fb (in Thousands)"
    ]],
    on="party_id",
    how="left"
)

In [79]:
df_model_base["Top_leader_fb (in Thousands)"] = df_model_base["Top_leader_fb (in Thousands)"].fillna(1)

In [ ]:
df_model_base["Facebook_Presence (In Thousands)"] = df_model_base["Facebook_Presence (In Thousands)"].fillna(1)

In [84]:
import numpy as np

df_model_base["facebook_log"] = np.log1p(
    df_model_base["Facebook_Presence (In Thousands)"]
)
df_model_base["leader_following_log"] = np.log1p(
    df_model_base["Top_leader_fb (in Thousands)"]
)

In [85]:
df_model_base.to_csv("data/processed/modeling_pr_base.csv", index=False)

In [150]:
base_2082 = party[["party_id"]][party["IS_ACTIVE?"] == "YES"]

In [151]:
districts = df_model_base[["district_id"]].drop_duplicates()

In [152]:
base_2082 = base_2082.merge(districts, how ="cross")

In [153]:
base_2082["election_year"] = 2082

In [1]:
base_2082.info()

NameError: name 'base_2082' is not defined

In [155]:
base_2082 = base_2082.merge(df_model_base[["party_id","district_id","vote_share"]], on = ["party_id","district_id"], how= "left" )

In [156]:
base_2082.rename(columns={"vote_share":"vote_share_lag"}, inplace=True)

In [157]:
base_2082["vote_share_lag"].fillna(0, inplace=True)

C:\Users\dipes\AppData\Local\Temp\ipykernel_18452\4207844022.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  base_2082["vote_share_lag"].fillna(0, inplace=True)


In [158]:
base_2082["is_new_party"] = np.where(base_2082["vote_share_lag"]==0, 1, 0)

In [159]:
base_2082 = base_2082.merge(
    df_model_base[["district_id","Absent rate within country 2078", "Absent rate abroad 2078", "Literacy Rate 2078", "Primary Education % 2078", "Lower Secondary % 2078", "Upper Secondary % 2078", "SLC or SEE % 2078", "Intermediate & equivalent % 2078", "Graduate & equivalent % 2078", "Post graduate equivalent & above % 2078", "female_ratio"]].drop_duplicates("district_id"), on="district_id", how = "left" )

In [165]:
base_2082 = base_2082.merge(
    party_lin[[
        "party_id",
        "IS_Major?",
        "IS_ALTERNATIVE_FORCE?",
        "GenZ_and_Youth_Favored?",
        "Was_Part_Of_Ousted_Government?"
    ]],
    on="party_id",
    how="left"
)

In [167]:
base_2082 = base_2082.merge(
    party[[
        "party_id",
        "Facebook_Presence (In Thousands)", 
        "Top_leader_fb (in Thousands)"
    ]],
    on="party_id",
    how="left"
)

In [168]:
base_2082["Top_leader_fb (in Thousands)"] = base_2082["Top_leader_fb (in Thousands)"].fillna(1)

In [169]:
base_2082["Facebook_Presence (In Thousands)"] = base_2082["Facebook_Presence (In Thousands)"].fillna(1)

In [170]:
base_2082["facebook_log"] = np.log1p(
    base_2082["Facebook_Presence (In Thousands)"]
)
base_2082["leader_following_log"] = np.log1p(
    base_2082["Top_leader_fb (in Thousands)"]
)

In [171]:
base_2082.head()

,party_id,district_id,election_year,vote_share_lag,is_new_party,Absent rate within country 2078,Absent rate abroad 2078,Literacy Rate 2078,Primary Education % 2078,Lower Secondary % 2078,...,Post graduate equivalent & above % 2078,female_ratio,IS_Major?,IS_ALTERNATIVE_FORCE?,GenZ_and_Youth_Favored?,Was_Part_Of_Ousted_Government?,Facebook_Presence (In Thousands),Top_leader_fb (in Thousands),facebook_log,leader_following_log
0,P001,1,2082,42.369621,0,7.14,5.17,82.21,30.53,21.99,...,1.07,0.496036,1,0,0,1,374.0,710.0,5.926926,6.566672
1,P001,2,2082,37.357759,0,9.86,6.79,82.29,28.29,21.87,...,0.98,0.502999,1,0,0,1,374.0,710.0,5.926926,6.566672
2,P001,3,2082,38.043496,0,6.89,6.74,83.37,27.97,21.81,...,1.30,0.501202,1,0,0,1,374.0,710.0,5.926926,6.566672
3,P001,4,2082,30.715977,0,3.86,9.15,82.83,25.65,19.71,...,1.54,0.520558,1,0,0,1,374.0,710.0,5.926926,6.566672
4,P001,5,2082,40.478568,0,12.11,5.00,79.70,33.99,22.99,...,1.11,0.496466,1,0,0,1,374.0,710.0,5.926926,6.566672


In [172]:
base_2082.to_csv("data/processed/base_pr_2082.csv", index = False)

In [176]:
party_79 = pd.read_csv("data/raw/Party_Database_79.csv",encoding = "utf-8-sig")

In [177]:
training_base =  pd.read_csv("data/processed/modeling_pr_base.csv",encoding = "utf-8-sig")

In [182]:
training_base.drop(columns=["IS_Major?", "Facebook_Presence (In Thousands)", "Top_leader_fb (in Thousands)", "facebook_log", "leader_following_log"], inplace=True)

In [184]:
training_base = training_base.merge(party_79[["party_id", "IS_Major?", "Facebook_Presence (In Thousands)", "Top_leader_fb (in Thousands)"]], on="party_id", how="left")

In [188]:
training_base["Facebook_Presence (In Thousands)"] = training_base["Facebook_Presence (In Thousands)"].fillna(1)

In [190]:
training_base["Top_leader_fb (in Thousands)"] = training_base["Top_leader_fb (in Thousands)"].fillna(1)

In [192]:
training_base.head()

,district_id,party_id,election_year,vote_share,vote_share_lag,vote_share_change,is_new_party,Absent rate within country 2078,Absent rate abroad 2078,Literacy Rate 2078,...,Intermediate & equivalent % 2078,Graduate & equivalent % 2078,Post graduate equivalent & above % 2078,female_ratio,IS_ALTERNATIVE_FORCE?,GenZ_and_Youth_Favored?,Was_Part_Of_Ousted_Government?,IS_Major?,Facebook_Presence (In Thousands),Top_leader_fb (in Thousands)
0,1,P001,2079,42.369621,44.075145,-1.705523,0,7.14,5.17,82.21,...,9.82,2.58,1.07,0.496036,0,0,1,1,350.0,650.0
1,1,P002,2079,34.007651,31.449762,2.557889,0,7.14,5.17,82.21,...,9.82,2.58,1.07,0.496036,0,0,1,1,300.0,621.0
2,1,P003,2079,11.156498,13.475434,-2.318936,0,7.14,5.17,82.21,...,9.82,2.58,1.07,0.496036,0,0,0,1,160.0,700.0
3,1,P004,2079,2.113726,0.000000,0.000000,1,7.14,5.17,82.21,...,9.82,2.58,1.07,0.496036,1,1,0,0,250.0,1200.0
4,1,P005,2079,1.376732,0.947807,0.428925,0,7.14,5.17,82.21,...,9.82,2.58,1.07,0.496036,0,0,0,0,40.0,850.0


In [193]:
training_base.loc[training_base["party_id"].isin(["P001", "P002", "P003"]), "Was_Part_Of_Ousted_Government?"] = 1


In [196]:
training_base.to_csv("data/processed/training_pr_base.csv", index=False)

In [180]:
party_79["IS_Major?"] = party_79["IS_Major"].map({"YES": 1, "NO": 0})


In [181]:
party_79.head()

,party_id,Party Name,English Name,Symbol,PartyName_2079,PartyName_2074,Merged_Into,IS_ACTIVE?,Leader,Ideology,Facebook_Presence (In Thousands),IS_NEW?,IS_RECENT?,IS_Major,IS_POPULAR,IS_ALTERNATIVE_FORCE?,Top Popular Leaders,Top_leader_fb (in Thousands),IS_Major?
0,P001,नेपाल कम्युनिष्ट पार्टी (एकीकृत मार्क्सवादी ले...,CPN (UML),सूर्य,नेपाल कम्युनिष्ट पार्टी (एकीकृत मार्क्सवादी ले...,नेपाल कम्युनिष्ट पार्टी (एमाले),जनशक्ति नेपाल\tआमूल परिवर्तन मसिहा पार्टी नेपा...,YES,K.P Sharma OLI,"Communism, Marxism–Leninism",350.0,NO,NO,YES,YES,NO,K P Sharma Oli | Mahesh Basnet | Shankar Pokh...,650.0,1
1,P002,नेपाली काँग्रेस,Nepali Congress,रूख,नेपाली काँग्रेस,नेपाली काँग्रेस,NaN,YES,Gagan Kumar Thapa,"Social democracy, Democratic socialism",300.0,NO,NO,YES,YES,NO,Sher Bahadur Deuba | Gagan Thapa | Pradip Paud...,621.0,1
2,P003,नेपाली कम्युनिष्ट पार्टी,Nepali Communist Party,पाँचकुने तारा,एकल चिन्ह गोलाकारभित्र हँसिया हथौडा (नेपाल कम्...,एकल चिन्ह गोलाकारभित्रको हसिया हथौडा लिएका,NaN,YES,Pushpa Kamal Dahal,"Communism, Marxism–Leninism, Maoism",160.0,NO,NO,YES,YES,NO,Pushpa Kamal Dahal (Prachanda) | Narayan Kaji ...,700.0,1
3,P004,राष्ट्रिय स्वतन्त्र पार्टी,Rastriya Swatantra Party,घण्टी,राष्ट्रिय स्वतन्त्र पार्टी,NaN,NaN,YES,Rabi Lamichhane,"Economic liberalism,Market Socialism, Progress...",250.0,YES,NO,NO,YES,YES,Rabi Lamichhane | Balen |Swarnim Wagle | DP Ar...,1200.0,0
4,P005,राष्ट्रिय प्रजातन्त्र पार्टी,Rastriya Prajatantra Party,हलो,राष्ट्रिय प्रजातन्त्र पार्टी,राष्ट्रिय प्रजातन्त्र पार्टी,NaN,YES,Rajendra Lingden,"Hindu nationalism, Monarchism, Conservatism",40.0,NO,NO,NO,YES,NO,Rajendra Lingden | Gyanendra Shahi | Rabindra ...,850.0,0
